<a href="https://colab.research.google.com/github/AlexNedyalkov/AladdinPerssonPyTorchTuorial/blob/main/02_Simple_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
import torchvision
from torchvision import transforms
import torchvision.datasets
from torch.utils.data import DataLoader

import os

In [2]:
batch_size = 32
lr = 0.001
epochs = 10
num_classes = 10

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_workers = os.cpu_count()
num_workers

2

In [4]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)


    model.train()
    return num_correct/num_samples

# 1. Get the data

In [5]:
train_dataset = torchvision.datasets.MNIST(train = True,
                                           root = 'date',
                                           transform  = transforms.ToTensor(),
                                           download = True)

test_dataset = torchvision.datasets.MNIST(train = False,
                                           root = 'date',
                                           transform  = transforms.ToTensor(),
                                           download = True)

train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, drop_last = True, num_workers = num_workers)
test_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, drop_last = True, num_workers = num_workers)


In [6]:
X, y = next(iter(train_loader))
in_channels = X.shape[1]

# 2. Model

In [7]:
class CNN(nn.Module):

  def __init__(self,in_channels) -> None:
    super().__init__()
    self.conv1 = torch.nn.Conv2d(in_channels = in_channels,
                                 out_channels = 10,
                                 kernel_size = (3,3),
                                 stride = 1,
                                 padding = 1)
    self.maxpool = torch.nn.MaxPool2d(kernel_size=(4,4))
    self.flatten = torch.nn.Flatten()
    self.linear = torch.nn.Linear(490, num_classes)

  def forward(self, X):
    X = self.conv1(X)
    X =self.maxpool(X)
    X = self.flatten(X)
    X = self.linear(X)
    return X
  

# 3. Initiate model, loss and optim

In [8]:
model = CNN(in_channels).to(device)
model(X).shape

torch.Size([32, 10])

In [9]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# 4. Train model

In [10]:
for epoch in range(epochs):
    print(epoch)
    for batch_idx, (data, targets) in enumerate((train_loader)):
        
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()


0
1
2
3
4
5
6
7
8
9


# 6. Check Accuracy

In [11]:
print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 98.26
Accuracy on test set: 98.26
